In [16]:
import pickle 

processed_doc = pickle.load(open('spacy_cleansing_of_paras.pkl','rb'))
wordSet = processed_doc['wordSet']
doc = processed_doc['clean_text_doclist']

doc_set_all = []
for docc in doc:
	doc_set = sorted(set().union(docc))
	doc_dict = dict.fromkeys(doc_set, 0)
	for word in docc:
		doc_dict[word] += 1 
	doc_set_all.append(doc_dict.copy())

# print ("wordSet  :   "," ".join(sorted(wordSet)))

In [17]:
def computeTF(tfDict,doc):
    doc_count = len(doc)
    for word, count in tfDict.items():
    	tfDict[word] = round((count+1)/doc_count,4)
    print(str(tfDict),'\n')
    return tfDict

tfdoc = []
for i in range(len(doc)):
    tfdoc.append(computeTF(doc_set_all[i], doc[i]))


{'France': 0.1333, 'Gandhi': 0.1333, 'India': 0.1333, 'Rafale': 0.1333, 'Rahul': 0.1333, 'advantage': 0.1333, 'business': 0.1333, 'crore': 0.1333, 'deal': 0.1333, 'fighter': 0.1333, 'house': 0.1333, 'indian': 0.1333, 'jet': 0.1333, 'result': 0.1333, 'say': 0.1333} 

{'Ganga': 0.0625, 'Golay': 0.0625, 'Governor': 0.0625, 'Krantikari': 0.0625, 'Morcha': 0.0625, 'P': 0.0625, 'Prasad': 0.0625, 'S': 0.0625, 'SKM': 0.0938, 'Saturday': 0.0625, 'Sikkim': 0.0625, 'assembly': 0.0625, 'call': 0.0625, 'chief': 0.0625, 'claim': 0.0625, 'delegation': 0.0625, 'form': 0.0625, 'government': 0.0625, 'lead': 0.0625, 'leader': 0.0625, 'legislative': 0.0625, 'legislature': 0.0625, 'lipped': 0.0625, 'majority': 0.0625, 'party': 0.0625, 'president': 0.0625, 'slend': 0.0625, 'stake': 0.0625, 'state': 0.0625, 'tight': 0.0625, 'win': 0.0625} 

{'Bhawan': 0.0444, 'Golay': 0.0667, 'Governor': 0.0444, 'Prasad': 0.0444, 'Raj': 0.0444, 'SDF': 0.0444, 'SKM': 0.0444, 'Sikkim': 0.0444, 'assembly': 0.0444, 'authorize': 

In [18]:
def computeIDF(doc_set_all,wordSet):
    import math
    idfDict = dict.fromkeys(wordSet, 0)
    N = len(doc_set_all)
    for word in wordSet:
        for i in range(N):
        	if word in doc_set_all[i].keys():
        		idfDict[word] += 1
             
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / val)
                        
    return idfDict        


idfs = computeIDF(doc_set_all,wordSet)

In [19]:
def computeTFIDF(tfdoc, idfs):
    tfidf = {}
    for word, val in tfdoc.items():
        tfidf[word] = round(val*idfs[word],4) #------rounded for error handeling of data(float64)
    print(str(tfidf),'\n')
    return tfidf

tfidf = []
for t in tfdoc:
    tfidf.append(computeTFIDF(t, idfs))

{'France': 0.106, 'Gandhi': 0.0782, 'India': 0.0818, 'Rafale': 0.0313, 'Rahul': 0.0964, 'advantage': 0.2358, 'business': 0.2759, 'crore': 0.1215, 'deal': 0.0368, 'fighter': 0.1092, 'house': 0.2487, 'indian': 0.135, 'jet': 0.0842, 'result': 0.2302, 'say': 0.0348} 

{'Ganga': 0.1432, 'Golay': 0.1432, 'Governor': 0.1432, 'Krantikari': 0.173, 'Morcha': 0.173, 'P': 0.1294, 'Prasad': 0.1166, 'S': 0.1244, 'SKM': 0.2149, 'Saturday': 0.1294, 'Sikkim': 0.1432, 'assembly': 0.1134, 'call': 0.0891, 'chief': 0.0758, 'claim': 0.0518, 'delegation': 0.1542, 'form': 0.0978, 'government': 0.0237, 'lead': 0.0758, 'leader': 0.0703, 'legislative': 0.1294, 'legislature': 0.1542, 'lipped': 0.173, 'majority': 0.1294, 'party': 0.0558, 'president': 0.0628, 'slend': 0.173, 'stake': 0.1294, 'state': 0.0658, 'tight': 0.173, 'win': 0.1166} 

{'Bhawan': 0.1229, 'Golay': 0.1528, 'Governor': 0.1017, 'Prasad': 0.0828, 'Raj': 0.1096, 'SDF': 0.1096, 'SKM': 0.1017, 'Sikkim': 0.1017, 'assembly': 0.0806, 'authorize': 0.1229,

In [20]:
tfidf_wordSet = [dict.fromkeys(wordSet, 0)]*len(doc)

# these two for loops aren't working the way they are supposed to:
# 1)
#         for i in range(len(doc)):
# 	          tfidf_wordSet[i].update(**tfidf[i])
# 2)
#         for i in range(len(doc)):
# 	          temp = tfidf[i]
# 	          tfidf_wordSet[i].update(**temp)
# 
# ------------ the solution ------------------
#  
#         for i in range(len(doc)):
# 	          temp = tfidf[i]
# 	          tfidf_wordSet[i] = dict(tfidf_wordSet[i],**temp)




i=0 
tfidf_final_struc = [] 
for wrdF in tfidf_wordSet: 
    temp = tfidf[i] 
    # print(temp, wrdF) 
    tfidf_final_struc.append(dict(wrdF, **temp)) 
    i += 1

import pandas as pd
from pandas import ExcelWriter
df = pd.DataFrame(tfidf_final_struc)

df.to_csv('tfidf_of_paras.csv') 